In [1]:
import pandas as pd
import numpy as np

import practice_code as pc
import individual_stats as ind


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
clutch = pd.read_csv('All seasons for comparison/clutch_time.csv')

In [3]:
clutch = pc.add_columns(clutch)

In [4]:
col_to_remove = ['a1', 'a2', 'a3','a4','a5','h1','h2','h3','h4','h5', 'play_length', 'entered', 'left', 'possession', 'shot_distance', 'original_x', 'original_y', 'converted_x', 'converted_y', 'num', 'away', 'home', 'outof', 'opponent', 'reason', 'elapsed', 'play_id']
clutch = pc.remove_col(clutch, col_to_remove)

In [5]:
clutch = pc.remove_junk_rows(clutch, 'event_type')

In [6]:
clutch = pc.replace_nan(clutch)

In [154]:
clutch.head()

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,h3,h4,h5,period,away_score,home_score,remaining_time,elapsed,play_length,play_id,team,event_type,assist,away,home,block,entered,left,num,opponent,outof,player,points,possession,reason,result,steal,type,shot_distance,original_x,original_y,converted_x,converted_y,description,pts_difference,winner,shots_missed,total_rebound,FT_made,off_rebound,FT_missed,total_blocks,assist_count,shots_made,3pt,3pt_shots,Dunk/Layup,Dunk/Layup_attempts,2pt_med/hard,med/hard_attempts,2fg_attempts
0,"=""0020400001""",2004-2005 Regular Season,2004-11-02,Jim Jackson,Tracy McGrady,Yao Ming,Charlie Ward,Maurice Taylor,Rasheed Wallace,Richard Hamilton,Ben Wallace,Tayshaun Prince,Chauncey Billups,4,77,83,00:01:41,00:10:19,00:00:00,435,HOU,free throw,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,Maurice Taylor,1.0,NaN,NaN,made,NaN,Free Throw 2 of 2,NaN,NaN,NaN,NaN,NaN,Taylor Free Throw 2 of 2 (10 PTS),-6,NaN,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0
1,"=""0020400001""",2004-2005 Regular Season,2004-11-02,Jim Jackson,Tracy McGrady,Yao Ming,Charlie Ward,Maurice Taylor,Rasheed Wallace,Richard Hamilton,Ben Wallace,Tayshaun Prince,Chauncey Billups,4,77,83,00:01:21,00:10:39,00:00:20,436,DET,miss,NaN,NaN,NaN,Tracy McGrady,NaN,NaN,NaN,NaN,NaN,Tayshaun Prince,0.0,NaN,NaN,missed,NaN,Layup,0.0,0,0,25,89,"McGrady BLOCK (1 BLK), MISS Prince Layup",-6,NaN,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1
2,"=""0020400001""",2004-2005 Regular Season,2004-11-02,Jim Jackson,Tracy McGrady,Yao Ming,Charlie Ward,Maurice Taylor,Rasheed Wallace,Richard Hamilton,Ben Wallace,Tayshaun Prince,Chauncey Billups,4,77,83,00:01:20,00:10:40,00:00:01,437,DET,rebound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tayshaun Prince,NaN,NaN,NaN,NaN,NaN,rebound offensive,NaN,NaN,NaN,NaN,NaN,Prince REBOUND (Off:1 Def:3),-6,NaN,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0
3,"=""0020400001""",2004-2005 Regular Season,2004-11-02,Jim Jackson,Tracy McGrady,Yao Ming,Charlie Ward,Maurice Taylor,Rasheed Wallace,Richard Hamilton,Ben Wallace,Tayshaun Prince,Chauncey Billups,4,79,85,00:00:58,00:11:02,00:00:12,443,HOU,shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Maurice Taylor,2.0,NaN,NaN,made,NaN,Jump Shot,17.0,136,100,11.4,15,Taylor 17' Jump Shot (12 PTS),-6,NaN,0,0,0,0,0,1,1,1,0,0,0,0,1,1,1
4,"=""0020400001""",2004-2005 Regular Season,2004-11-02,Jim Jackson,Tracy McGrady,Yao Ming,Charlie Ward,Maurice Taylor,Rasheed Wallace,Richard Hamilton,Ben Wallace,Tayshaun Prince,Chauncey Billups,4,79,85,00:00:40,00:11:20,00:00:18,444,DET,miss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rasheed Wallace,0.0,NaN,NaN,missed,NaN,Jump Shot,27.0,174,207,42.4,68.3,MISS Wallace 27' 3PT Jump Shot,-6,NaN,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0


In [17]:
clutch = pc.remove_not_imp_games(clutch, 'game_id')

# 3pt stats for all players

In [110]:
x = clutch[['player', '3pt', '3pt_shots']]
x = x[x['3pt_shots']==1]
threes = x.groupby('player').sum()
threes['3pt%'] = round((threes['3pt']/ threes['3pt_shots'])*100,2)
threes.reset_index(level=0, inplace=True)

# 2pt stats for all players

In [108]:
x = clutch[['player', '2pt_med/hard', 'Dunk/Layup', '2fg_attempts']]
x = x[x['2fg_attempts']==1]
twos = x.groupby('player').sum()
twos['2fg_made'] = twos['2pt_med/hard'] + twos['Dunk/Layup']
twos['2pt%'] = round((twos['2fg_made']/ twos['2fg_attempts'])*100,2)
twos = twos.drop(['2pt_med/hard', 'Dunk/Layup'], axis = 1)
twos.reset_index(level=0, inplace=True)

In [130]:
games = pc.get_total_games(clutch)
games.reset_index(level=0, inplace=True)
games = games.drop('index', axis = 1)


# Total Shots

In [179]:
x = clutch[['player', 'shots_missed', 'shots_made']]
x = x[(x['shots_missed'] ==1) | x['shots_made']==1]
x['total_shots'] = x['shots_missed'] + x['shots_made']
x = x.drop('shots_missed', axis =1)
total_fg = x.groupby('player').sum()
total_fg.reset_index(level=0, inplace=True)
total_fg['shooting%'] = round((total_fg['shots_made']/total_fg['total_shots'])*100,2)

# FT stats

In [180]:
x = clutch[['player', 'FT_missed', 'FT_made']]
x = x[(x['FT_missed'] ==1) | x['FT_made']==1]
x['FT_Attempts'] = x['FT_missed'] + x['FT_made']
x = x.drop('FT_missed', axis =1)
FT = x.groupby('player').sum()
FT.reset_index(level=0, inplace=True)
FT['FT%'] = round((FT['FT_made']/FT['FT_Attempts'])*100,2)

# off rebounds, assists, & block stats

In [83]:
clutch['assist_count'] = (clutch['assist']!='')*1
clutch['total_blocks'] = (clutch['block']!='')*1

In [111]:
x = clutch[['player', 'assist_count']]
x = x[x['assist_count']==1]
asts = x.groupby('player').sum()
asts.reset_index(level=0, inplace=True)

In [112]:
x = clutch[['player', 'total_blocks']]
x = x[x['total_blocks']==1]
blks = x.groupby('player').sum()
blks.reset_index(level=0, inplace=True)

In [113]:
x = clutch[['player', 'off_rebound']]
x = x[x['off_rebound']==1]
off_rebound = x.groupby('player').sum()
off_rebound.reset_index(level=0, inplace=True)

In [218]:
non_shooting_stats = pd.merge(pd.merge(asts, blks, on= 'player', how='outer'), off_rebound, on= 'player', how='outer')

In [219]:
shooting_stats = pd.merge(pd.merge(games, twos, on= 'player', how='outer'), threes, on= 'player', how='outer')

In [220]:
full_stats = pd.merge(pd.merge(shooting_stats, total_fg, on= 'player', how='outer'), FT, on= 'player', how='outer')

In [221]:
full_stats = pd.merge(full_stats, non_shooting_stats, on= 'player', how='outer')

In [222]:
full_stats = full_stats.fillna(0)

In [223]:
cols = ['2fg_attempts', '2fg_made', '3pt','3pt_shots', 'shots_made', 'total_shots', 'FT_made', 'FT_Attempts', 'assist_count', 'total_blocks','off_rebound']

In [224]:
def get_ints(df, col):
    for name in col:
        df[col] = df[col].astype(np.int64)
    return df

In [225]:
full_stats = get_ints(full_stats, cols)

In [227]:
full_stats.columns

Index(['player', 'total_games', '2pt_attempts', '2pt_made', '2pt%', '3pt_made',
       '3pt_attempts', '3pt%', 'shots_made', 'total_shots', 'shooting%',
       'FT_made', 'FT_Attempts', 'FT%', 'total_assists', 'total_blocks',
       'off_rebound'],
      dtype='object')

In [226]:
full_stats = full_stats.rename(columns={'2fg_attempts':'2pt_attempts', '2fg_made':'2pt_made', 'assist_count': 'total_assists', '3pt': '3pt_made', '3pt_shots': '3pt_attempts'})

In [228]:
full_stats = full_stats[['player', 'total_games', 'shots_made', 'total_shots', 'shooting%','2pt_made', '2pt_attempts', '2pt%', '3pt_made','3pt_attempts', '3pt%','FT_made', 'FT_Attempts', 'FT%', 'total_assists', 'total_blocks', 'off_rebound']]

In [230]:
full_stats.to_csv('stats_for_flask', encoding='utf-8', index=False)

In [7]:
import pickle

In [13]:
clutch_stats = pd.read_csv('All seasons for comparison/stats_for_flask')

In [14]:
#Get complete stats for a player
def player_stat(name1, name2, name3):
    player1 = (clutch_stats.loc[clutch_stats['player'] == name1]) 
    player2 = (clutch_stats.loc[clutch_stats['player'] == name2])
    player3 = (clutch_stats.loc[clutch_stats['player'] == name3])
    both = pd.merge(pd.merge(player1,player2, how = 'outer'), player3, how = 'outer')
    both = both.T
    both.reset_index(level=0, inplace=True)
    both.columns = both.iloc[0]
    stats = both.reindex(both.index.drop(0))
    stats = stats.rename(columns={'player':' '})
    stats = stats.set_index(' ')
    stats = stats.drop(['shots_made', '2pt_made', '3pt_made', 'FT_made'])
    return stats

In [9]:
f = player_stat('LeBron James', 'Joe Johnson', 'Kobe Bryant')


In [10]:
clutch_stats.head()

,player,total_games,shots_made,total_shots,shooting%,2pt_made,2pt_attempts,2pt%,3pt_made,3pt_attempts,3pt%,FT_made,FT_Attempts,FT%,total_assists,total_blocks,off_rebound
0,LeBron James,358,338,770,43.90,260,496,52.42,78,274,28.47,453,615,73.66,80,44,52
1,Dirk Nowitzki,341,249,607,41.02,198,445,44.49,51,162,31.48,320,362,88.40,126,21,37
2,Joe Johnson,309,237,604,39.24,164,398,41.21,73,206,35.44,221,267,82.77,80,30,23
3,Chris Paul,308,203,532,38.16,180,416,43.27,23,116,19.83,375,437,85.81,26,28,21
4,Carmelo Anthony,306,240,602,39.87,194,455,42.64,46,147,31.29,298,365,81.64,89,49,67


In [23]:
clutch_stats.total_shots.mean()

58.36944937833037

In [19]:
x = (clutch_stats.total_games > clutch_stats.total_games.mean())

In [20]:
x.value_counts()

False    760
True     366
Name: total_games, dtype: int64

In [25]:
q = clutch_stats.total_shots>clutch_stats.total_shots.mean()

In [26]:
q.value_counts()

False    809
True     317
Name: total_shots, dtype: int64

In [27]:
317/(317+809)

0.28152753108348133